# Malaria Detection using Deep Learning


https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

### Importing necessary libraries

In [1]:
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras.layers import Dense#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Flatten#used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D#Convolutional layer
from tensorflow.keras.layers import MaxPool2D# used for downsampling of the input
from tensorflow.keras.layers import Dropout #used for preventing overfitting of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator# to artificially expand the size of a training dataset
import numpy as np#used for numerical analysis

In [2]:
img_width=64 #setting default image width size 
img_height=64 #setting default image height size

### Image Data Agumentation

In [3]:
datagen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2) #Image data augmentation

In [9]:
train_data_generator=datagen.flow_from_directory(directory=r"C:\Users\chand\Downloads\MalariaDetection-DL-main-20220127T104713Z-001\MalariaDetection-DL-main\Dataset\cell_images\Train",
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="training")#applying image data agumentation to training data

Found 22048 images belonging to 2 classes.


In [12]:
validation_data_generator=datagen.flow_from_directory(directory=r"C:\Users\chand\Downloads\MalariaDetection-DL-main-20220127T104713Z-001\MalariaDetection-DL-main\Dataset\cell_images\Test",
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="validation")#applying image data agumentation to testing data

Found 5510 images belonging to 2 classes.


In [13]:
train_data_generator.labels #checking the labels

array([0, 0, 0, ..., 1, 1, 1])

### Creating the Model

In [14]:
model= Sequential() #our model

model.add(Conv2D(16,(3,3),input_shape=(img_width,img_height,3),activation="relu"))#convolutional layer
model.add(MaxPool2D(2,2)) # Downsamples the input
model.add(Dropout(0.2)) #drop input randomly for preventing from overfitting

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(64,activation="relu"))#deeply connected neural network layer
model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid")) #output layer with one neuron


In [15]:
model.summary() #summary of our neural network

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

### Compiling the model

In [16]:
#compile modelb
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

### Fitting the model

In [17]:
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

C:\Users\chand\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1378/1378 [==============================] - 551s 400ms/step - loss: 0.5394 - accuracy: 0.7235 - val_loss: 0.2230 - val_accuracy: 0.9267
Epoch 2/5
1378/1378 [==============================] - 466s 338ms/step - loss: 0.2346 - accuracy: 0.9183 - val_loss: 0.2211 - val_accuracy: 0.9270
Epoch 3/5
1378/1378 [==============================] - 253s 184ms/step - loss: 0.2132 - accuracy: 0.9339 - val_loss: 0.1765 - val_accuracy: 0.9370
Epoch 4/5
1378/1378 [==============================] - 60s 44ms/step - loss: 0.1899 - accuracy: 0.9404 - val_loss: 0.1721 - val_accuracy: 0.9397
Epoch 5/5
1378/1378 [==============================] - 60s 43ms/step - loss: 0.1738 - accuracy: 0.9455 - val_loss: 0.1636 - val_accuracy: 0.9439


### Saving our model

In [20]:
from tensorflow.keras.models import load_model

model.save('maleria.h5')

### Predicting our results

In [19]:
from tensorflow.keras.models import load_model
model = load_model(r'C:\Users\chand\maleria.h5') #loading our trained model

OSError: SavedModel file does not exist at: C:\Users\chand\maleria.h5\{saved_model.pbtxt|saved_model.pb}

In [8]:
y_pred = model.predict(validation_data_generator) #performing prediction to the test data

In [9]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)#Returns the indices of the maximum values along an axis
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Taking images as input an checking results

In [31]:
from tensorflow.keras.preprocessing import image
img=image.load_img(r'E:\Malaria Detection using DL\uninfected.png',target_size=(64,64))#loading image
x=image.img_to_array(img) #converting in to array formst
x.shape #checking its shape

(64, 64, 3)

In [32]:
x=np.expand_dims(x,axis=0) #changing its dimensions as per our requirement 
print(x.shape)
#img_data=x.reshape(1, 64, 64, 3)
#img_data.shape

(1, 64, 64, 3)


In [38]:
a=model.predict(x) #prediction

In [36]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")